In [1]:
import pandas as pd

In [2]:
train_data = pd.read_csv('train.csv')
test_data_orig = pd.read_csv('test.csv')
test_data_orig['original_index'] = test_data_orig.index
test_data_copy = test_data_orig.copy()

In [3]:
import roman

def roman_to_arabic(roman_string):
    return roman.fromRoman(roman_string)
    
train_data['B'] = train_data['B'].apply(roman_to_arabic)
test_data_copy['B'] = test_data_copy['B'].apply(roman_to_arabic)

In [4]:
x_train = train_data[['A', 'B', 'C', 'D', 'E', 'F']]

y_train = train_data['target']

In [5]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np

scaler = StandardScaler()
x_scaled = scaler.fit_transform(x_train)
x_train, x_val, y_train, y_val = train_test_split(x_scaled, y_train, test_size=0.2, random_state=42)

model = tf.keras.Sequential()

#входной слой
model.add(tf.keras.layers.InputLayer(input_shape=(x_train.shape[1],)))

#скрытые слои
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.1))

model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.1))

model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.1))

model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.1))

model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.1))

#выходной слой
model.add(tf.keras.layers.Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')

history = model.fit(x_train, y_train, epochs=1000, batch_size=512, validation_data=(x_val, y_val))

loss = model.evaluate(x_val, y_val)
print(f"{loss}")

Epoch 1/1000


C:\Users\Phili\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 225ms/step - loss: 1963.3573 - val_loss: 1886.2324
Epoch 2/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 1940.5256 - val_loss: 1882.3513
Epoch 3/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - loss: 1934.0698 - val_loss: 1878.3259
Epoch 4/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 1903.4607 - val_loss: 1875.1469
Epoch 5/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 1891.9351 - val_loss: 1872.7269
Epoch 6/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 1906.1611 - val_loss: 1870.9290
Epoch 7/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 1890.6842 - val_loss: 1869.8223
Epoch 8/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 1874.2217 - val_loss: 1869.9225
Epoch 9/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 1869.4172 - val_loss: 1870.8301
Epoch 10/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 1855.9543 - val_loss: 1872.8350
Epoch 11/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 1858.3668 - val_loss: 1875.7439
Epoch 12/

In [6]:
from sklearn.metrics import mean_squared_error

y_pred = model.predict(x_val)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print(f"{rmse}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step  
4.3302562887993


In [7]:
x_test = test_data_copy[['A', 'B', 'C', 'D', 'E', 'F']]

y_pred = model.predict(x_test)
test_data_copy['target'] = y_pred
test_data_orig['target'] = test_data_copy.sort_values('original_index')['target'].values
test_data_orig = test_data_orig.drop(columns=['original_index'])

test_data_orig.to_csv('answers_final.csv', index=False)

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
